In [1]:
%pip  install --upgrade transformers optimum

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -U wheel

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install --upgrade auto-gptq

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install -v flash-attn==2.1.1 --no-build-isolation

Using pip 23.3.2 from /home/radmad/LLMCert/.venv/lib/python3.10/site-packages/pip (python 3.10)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
print(torch.__version__)
print (torch.cuda.is_available())

2.1.2+cu121
True


In [7]:
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install --upgrade bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.


In [10]:
model_name = "mistralai/Mixtral-8x7B-v0.1"


In [11]:
from transformers import BitsAndBytesConfig
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)

In [12]:
from transformers import AutoModelForCausalLM, pipeline

model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
# # model_name_or_path = "TheBloke/dolphin-2.7-mixtral-8x7b-GPTQ"
# model_name_or_path = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# # To use a different branch, change revision
# # For example: revision="gptq-4bit-128g-actorder_True"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                                device_map="auto",
#                                                torch_dtype=torch.float16,
#                                                offload_folder="offload",
#                                                trust_remote_code=True,
#                                                low_cpu_mem_usage=True,
#                                                quantization_config=bnb_config)
                                             

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)



Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [13]:
# prompt = "Write a story about llamas"
# system_message = "You are a story writing assistant"
# prompt_template=f'''<|im_start|>system
# {system_message}<|im_end|>
# <|im_start|>user
# {prompt}<|im_end|>
# <|im_start|>assistant
# '''


In [14]:
from transformers import  AutoTokenizer

#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

In [31]:
prompt="Summarize your thoughts on AI hallucinations"

In [34]:
input_ids = tokenizer.encode(prompt, return_tensors='pt',padding=True).cuda()
output = model.generate(inputs=input_ids, pad_token_id=tokenizer.unk_token_id,do_sample=True,temperature=0.4, repetition_penalty=1.1, top_p=0.95, top_k=40, max_new_tokens=512).cuda()
print(tokenizer.decode(output[0]))

<s> Summarize your thoughts on AI hallucinations</s> of the future.

I think that it is a very interesting idea to consider what might happen if we were able to create artificial intelligence that could learn from its own experiences and then use this knowledge in order to predict events before they occur. This would be extremely useful for many different purposes, such as forecasting weather patterns or even stock market trends! It also has some potentially dangerous consequences though; if someone was able to control an entire population through their predictions about how people will act based off past behavior (which seems like something humans do anyway), then there’s no telling where things could go wrong…</s>


In [33]:
print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.unk_token_id,
    do_sample=True,
    temperature=0.4,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt)[0]['generated_text'])

*** Pipeline:


Summarize your thoughts on AI hallucinations and their impact on the reliability of AI-generated content.

## Conclusion

In conclusion, AI hallucinations are a significant concern in the field of AI-generated content. While AI models have made significant progress in generating high-quality content, they are not immune to hallucinations. These hallucinations can lead to inaccurate or misleading information, which can have serious consequences in fields such as healthcare, finance, and journalism.

To address this issue, it is important to develop AI models that are more robust and resistant to hallucinations. This can be achieved through techniques such as data augmentation, adversarial training, and model ensembling. Additionally, it is important to have human oversight and verification of AI-generated content to ensure its accuracy and reliability.

In the future, we can expect to see continued research and development in the field of AI-generated content, with a focus on addressing